In [12]:
import pandas as pd
import numpy as np
import sqlite3

In [41]:
df = pd.read_csv('/Users/serdarevichar/Library/CloudStorage/GoogleDrive-serdarevichar@gmail.com/My Drive/fantasy-football-database.csv', index_col = 'Unnamed: 0')
df['Home Team'] = df['Home Team'].replace({'The':'Klapp'})
df['Away Team'] = df['Away Team'].replace({'The':'Klapp'})

df.to_csv('database/fantasy-football-matchup-data.csv')

temp1 = df[['Year','Week','Playoff Flag','Home Team','Home Score','Away Score']].copy()
temp2 = df[['Year','Week','Playoff Flag','Away Team','Away Score','Home Score']].copy()

temp1.rename(columns={'Home Team':'Team', 'Home Score':'Score', 'Away Score':'Opp Score'}, inplace=True)
temp2.rename(columns={'Away Team':'Team', 'Away Score':'Score', 'Home Score':'Opp Score'}, inplace=True)

df_converted = pd.concat([temp1,temp2], axis=0, ignore_index=True)
df_converted['Win'] = df_converted['Score'] > df_converted['Opp Score']
df_converted['Win'] = df_converted['Win'].map(int)
df_converted

df_converted.sort_values(['Year','Week'], inplace = True)
df_converted.to_csv('database/fantasy-football-game-data.csv')
df_converted

,Year,Week,Playoff Flag,Team,Score,Opp Score,Win
0,2019,1,False,Justin,158.02,113.20,1
1,2019,1,False,Dante,107.92,126.14,0
2,2019,1,False,Michael,112.68,134.32,0
3,2019,1,False,Andrew,128.54,141.26,0
4,2019,1,False,Haris,135.80,101.82,1
...,...,...,...,...,...,...,...
495,2024,16,True,Andrew,117.28,134.34,0
991,2024,16,True,Zach,139.88,136.02,1
992,2024,16,True,Noah,134.34,117.28,1
496,2024,17,True,Zach,128.60,89.22,1


In [49]:
df = pd.read_csv('/Users/serdarevichar/Library/CloudStorage/GoogleDrive-serdarevichar@gmail.com/My Drive/fantasy-football-database.csv', index_col = 'Unnamed: 0')
df['Home Team'] = df['Home Team'].replace({'The':'Klapp'})
df['Away Team'] = df['Away Team'].replace({'The':'Klapp'})

temp1 = df[['Year','Week','Playoff Flag','Home Team','Home Score','Away Score']].copy()
temp2 = df[['Year','Week','Playoff Flag','Away Team','Away Score','Home Score']].copy()

temp1.rename(columns={'Home Team':'Team', 'Home Score':'Score', 'Away Score':'Opp Score'}, inplace=True)
temp2.rename(columns={'Away Team':'Team', 'Away Score':'Score', 'Home Score':'Opp Score'}, inplace=True)

df_converted = pd.concat([temp1,temp2], axis=0, ignore_index=True)
df_converted['Result'] = (df_converted['Score'] > df_converted['Opp Score']).map({True:'Win', False:'Loss'})
df_converted

,Year,Week,Playoff Flag,Team,Score,Opp Score,Result
0,2019,1,False,Justin,158.02,113.20,Win
1,2019,1,False,Dante,107.92,126.14,Loss
2,2019,1,False,Michael,112.68,134.32,Loss
3,2019,1,False,Andrew,128.54,141.26,Loss
4,2019,1,False,Haris,135.80,101.82,Win
...,...,...,...,...,...,...,...
989,2024,15,True,Zach,153.82,150.24,Win
990,2024,15,True,Noah,119.10,105.18,Win
991,2024,16,True,Zach,139.88,136.02,Win
992,2024,16,True,Noah,134.34,117.28,Win


In [4]:
years = df_converted['Year'].unique()
years_weeks = [(year, df_converted.loc[(df_converted['Year'] == year) & (df_converted['Playoff Flag'] == False), 'Week'].max()) for year in years]
years_weeks

[(2019, 13), (2020, 13), (2021, 14), (2022, 14), (2023, 14), (2024, 14)]

# Weekly Scoreboard Table





In [266]:
week = df.loc[(df['Year'] == 2019) & (df['Week'] == 1)]
week

,Year,Week,Playoff Flag,Home Team,Home Score,Home Projection,Away Team,Away Score,Away Projection
0,2019,1,False,Justin,158.02,115.55,Klapp,113.20,121.01
1,2019,1,False,Dante,107.92,118.65,Zach,126.14,114.46
2,2019,1,False,Michael,112.68,118.23,McGwire,134.32,122.20
3,2019,1,False,Andrew,128.54,121.59,Carter,141.26,117.11
4,2019,1,False,Haris,135.80,119.44,Ethan,101.82,112.52
5,2019,1,False,Noah,153.86,125.17,Nathan,132.22,116.77


In [7]:
scoreboard = week[['Home Team','Home Score','Away Score','Away Team']].copy()
scoreboard

,Home Team,Home Score,Away Score,Away Team
0,Justin,158.02,113.20,The
1,Dante,107.92,126.14,Zach
2,Michael,112.68,134.32,McGwire
3,Andrew,128.54,141.26,Carter
4,Haris,135.80,101.82,Ethan
5,Noah,153.86,132.22,Nathan


# Season Standings Table

In [235]:
year = 2022
week = 14

temp = df_converted.loc[(df_converted['Year'] == year) & (df_converted['Week'] <= week)]

teams = temp['Team'].unique()
records = []

for team in teams:
    temp_team = temp.loc[temp['Team'] == team]
    wins = temp_team['Win'].sum()
    losses = week - wins
    record = f'{wins}-{losses}'

    pf = round(temp_team['Score'].sum(), 2)
    avg_pf = round(pf / week, 2)
    pa = round(temp_team['Opp Score'].sum(), 2)
    avg_margin = round((pf - pa) / week, 2)

    records.append([team, wins, record, pf, pa, avg_pf, avg_margin])

records

[['Haris', 7, '7-7', 1647.48, 1579.54, 117.68, 4.85],
 ['Ethan', 8, '8-6', 1463.36, 1486.96, 104.53, -1.69],
 ['Dante', 10, '10-4', 1747.0, 1572.08, 124.79, 12.49],
 ['Andrew', 8, '8-6', 1599.88, 1650.06, 114.28, -3.58],
 ['Carter', 8, '8-6', 1611.1, 1444.86, 115.08, 11.87],
 ['Zach', 6, '6-8', 1531.06, 1668.74, 109.36, -9.83],
 ['Kevin', 8, '8-6', 1623.22, 1687.58, 115.94, -4.6],
 ['Michael', 7, '7-7', 1560.4, 1644.98, 111.46, -6.04],
 ['Tyler', 4, '4-10', 1396.3, 1577.24, 99.74, -12.92],
 ['Noah ', 7, '7-7', 1649.96, 1522.02, 117.85, 9.14],
 ['Nathan', 4, '4-10', 1708.72, 1659.74, 122.05, 3.5],
 ['McGwire', 7, '7-7', 1487.38, 1532.06, 106.24, -3.19]]

In [239]:
table = pd.DataFrame(records, columns=['Team','Wins','Record', 'Points For','Points Against','Avg Points For','Avg Margin'])
table.sort_values(['Wins','Points For'], ascending=False, ignore_index=True, inplace=True)
table[['Team','Record','Points For','Points Against','Avg Points For','Avg Margin']]

,Team,Record,Points For,Points Against,Avg Points For,Avg Margin
0,Dante,10-4,1747.00,1572.08,124.79,12.49
1,Kevin,8-6,1623.22,1687.58,115.94,-4.60
2,Carter,8-6,1611.10,1444.86,115.08,11.87
3,Andrew,8-6,1599.88,1650.06,114.28,-3.58
4,Ethan,8-6,1463.36,1486.96,104.53,-1.69
5,Noah,7-7,1649.96,1522.02,117.85,9.14
6,Haris,7-7,1647.48,1579.54,117.68,4.85
7,Michael,7-7,1560.40,1644.98,111.46,-6.04
8,McGwire,7-7,1487.38,1532.06,106.24,-3.19
9,Zach,6-8,1531.06,1668.74,109.36,-9.83


# Luck Score Table




In [125]:
standings_data.head()

,Team,Score,Opp Score,Win
0,Justin,158.02,113.20,1
1,Dante,107.92,126.14,0
2,Michael,112.68,134.32,0
3,Andrew,128.54,141.26,0
4,Haris,135.80,101.82,1


In [156]:
mean = standings_data['Score'].mean()
std = standings_data['Score'].std() * 0.5

print(mean,std)

119.09870370370369 11.986959368542161


In [76]:
def opp_luck_score(score, mean=None, std=None):
    return int(((mean - score) / np.absolute(mean - score)) * np.floor(np.absolute(mean - score) / std))

In [157]:
opp_luck_score(100,mean=mean,std=std)

1

In [158]:
standings_data['Opp Luck Score'] = standings_data[['Opp Score']].map(opp_luck_score, mean=mean, std=std)
standings_data

,Team,Score,Opp Score,Win,Opp Luck Score,Your Luck Score
0,Justin,158.02,113.20,1,0,0
1,Dante,107.92,126.14,0,0,0
2,Michael,112.68,134.32,0,-1,0
3,Andrew,128.54,141.26,0,-1,0
4,Haris,135.80,101.82,1,1,0
...,...,...,...,...,...,...
103,Michael,108.56,109.78,0,0,0
104,Dante,117.50,133.12,0,-1,0
105,Justin,115.30,122.36,0,0,0
106,Zach,115.80,100.66,1,1,0


In [139]:
def your_luck_score(score, result, mean=None, std=None):
    xi = lambda x: np.floor((mean + x) / (2*mean)) 
    return int(np.floor(np.absolute(mean - score) / std) * (result * xi(2*mean - score) - (1-result) * xi(score)))

In [141]:
your_luck_score(175,0, mean=mean, std=std)

-2

In [162]:
yls = [your_luck_score(score, result, mean=mean, std=std) for score, result in standings_data[['Score','Win']].values]
standings_data['Your Luck Score'] = yls
standings_data.sort_values('Score', ascending=False).head(10)

,Team,Score,Opp Score,Win,Opp Luck Score,Your Luck Score
30,The,183.40,122.04,1,0,0
70,Noah,181.58,108.72,1,0,0
83,Michael,172.26,172.16,1,-4,0
29,Dante,172.16,172.26,0,-4,-4
28,Justin,164.64,101.92,1,1,0
25,Carter,163.82,114.76,1,0,0
77,Michael,162.32,146.48,1,-2,0
48,Andrew,161.64,137.32,1,-1,0
0,Justin,158.02,113.20,1,0,0
43,Andrew,155.50,108.74,1,0,0


In [164]:
def close_luck_score(score,opp_score):
    if (score - opp_score) < 3 and (score - opp_score) > 0:
        return 1
    elif (score - opp_score) < 0 and (score - opp_score) > -3:
        return -1
    else:
        return 0    

In [171]:
cls = [close_luck_score(score, opp_score) for score, opp_score in standings_data[['Score','Opp Score']].values]
standings_data['Close Luck Score'] = cls
standings_data['Luck Score'] = standings_data[['Opp Luck Score','Your Luck Score','Close Luck Score']].sum(axis=1)


# Champions Table

In [211]:
champions = df_converted.loc[(df_converted['Week'] == 17) & (df_converted['Win'] == 1), ['Year','Team']].values
champions

array([[2019, 'Dante'],
       [2020, 'Ethan'],
       [2021, 'Andrew'],
       [2022, 'Dante'],
       [2023, 'Haris'],
       [2024, 'Zach']], dtype=object)

In [226]:
data = []
for year, team in champions:
    temp = df_converted.loc[(df_converted['Year'] == year) & (df_converted['Team'] == team) & (df_converted['Playoff Flag'] == False)]
    wins = temp['Win'].sum()
    losses = len(temp) - wins
    record = f'{wins}-{losses}'

    pf = round(temp['Score'].sum(), 2)
    avg_pf = round(pf / len(temp), 2)
    pa = round(temp['Opp Score'].sum(), 2)
    avg_margin = round((pf - pa) / len(temp), 2)

    data.append([year, team, record, pf, avg_pf, avg_margin])
    
pd.DataFrame(data, columns=['Year','Team','Record','Points For','Avg Points For','Avg Margin'])

,Year,Team,Record,Points For,Avg Points For,Avg Margin
0,2019,Dante,8-5,1612.06,124.00,9.25
1,2020,Ethan,11-2,1639.56,126.12,22.44
2,2021,Andrew,10-4,1901.54,135.82,10.45
3,2022,Dante,10-4,1747.00,124.79,12.49
4,2023,Haris,12-2,1801.34,128.67,24.65
5,2024,Zach,8-6,1762.04,125.86,6.56


In [232]:
years = np.arange(2019,2025)
for year in years:
    if year < 2021:
        weeks = np.arange(1,14)
    else:
        weeks = np.arange(1,15)
    print(weeks)

[ 1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]
[ 1  2  3  4  5  6  7  8  9 10 11 12 13 14]


# Individual Team Table

In [43]:
teams = df_converted['Team'].unique()
team = 'Kevin'

temp = df_converted.loc[(df_converted['Team'] == team) & (df_converted['Playoff Flag'] == False)]
years = temp['Year'].unique()

data = []
for year in years:
    temp_year = temp.loc[temp['Year'] == year]

    wins = temp_year['Win'].sum()
    losses = len(temp_year) - wins
    record = f'{wins}-{losses}'

    pf = round(temp_year['Score'].sum(), 2)
    avg_pf = round(pf / len(temp_year), 2)
    pa = round(temp_year['Opp Score'].sum(), 2)
    avg_margin = round((pf - pa) / len(temp_year), 2)

    data.append([year, team, record, pf, pa, avg_pf, avg_margin])

pd.DataFrame(data, columns=['Season','Team','Record','Points For','Points Against','Avg Points For','Avg Margin'])

,Season,Team,Record,Points For,Points Against,Avg Points For,Avg Margin
0,2022,Kevin,8-6,1623.22,1687.58,115.94,-4.60
1,2023,Kevin,7-7,1800.40,1696.42,128.60,7.43
2,2024,Kevin,11-3,1777.24,1568.64,126.95,14.90


In [38]:
conn = sqlite3.connect('database/fantasy-football.db')

query = '''
        SELECT
            Year,
            Record,
            Ranking,
            ROUND("Points For", 2) AS "Points For",
            ROUND("Points Against", 2) AS "Points Against",
            "Avg Points For",
            "Avg Margin"
        FROM season_totals
    '''

data = pd.read_sql(query + f"WHERE team = '{team}'", con=conn)

conn.close()

total_pf = data['Points For'].sum()
total_pa = data['Points Against'].sum()
avg_ranking = round(data['Ranking'].mean(), 2)
#total_avg_margin = (total_pf - total_pa) / 

data.loc['Total'] = [
    '',
    '',
    avg_ranking,
    total_pf,
    total_pa,
    '',
    ''
]

data

,Year,Record,Ranking,Points For,Points Against,Avg Points For,Avg Margin
0,2019,7-6,5.0,1548.30,1483.26,119.1,5.0
1,2020,3-10,10.0,1440.82,1773.88,110.83,-25.62
2,2021,8-6,3.0,1855.22,1655.12,132.52,14.29
3,2022,7-7,7.0,1647.48,1579.54,117.68,4.85
4,2023,12-2,1.0,1801.34,1456.30,128.67,24.65
5,2024,3-11,10.0,1468.60,1616.88,104.9,-10.59
Total,,,6.0,9761.76,9564.98,,


In [9]:
query = '''
    SELECT *
    FROM results
'''

query += 'WHERE week = 4 AND year = 2022'

conn = sqlite3.connect('database/fantasy-football.db')

df = pd.read_sql(query, con=conn)

conn.close()

print(df)


    index  Year  Week  Playoff Flag     Team   Score  Opp Score Result
0     248  2022     4             0    Kevin  127.24     113.96    Win
1     249  2022     4             0   Nathan  125.08      85.28    Win
2     250  2022     4             0  McGwire   79.20     107.86   Loss
3     251  2022     4             0     Zach  150.22     133.80    Win
4     252  2022     4             0   Carter  102.46     107.50   Loss
5     253  2022     4             0   Andrew  134.68     180.24   Loss
6     745  2022     4             0    Noah   113.96     127.24   Loss
7     746  2022     4             0    Tyler   85.28     125.08   Loss
8     747  2022     4             0  Michael  107.86      79.20    Win
9     748  2022     4             0    Haris  133.80     150.22   Loss
10    749  2022     4             0    Ethan  107.50     102.46    Win
11    750  2022     4             0    Dante  180.24     134.68    Win


# Requests to Github page + API

In [6]:
import requests
from bs4 import BeautifulSoup

In [21]:
df_converted.to_json('api/data.json', orient='records', indent=2)

In [23]:
url = 'https://serdarevichar8.github.io/fantasy-football-website/api/data.json'

r = requests.get(url)

if not r.ok:
    print(r.status_code)

print(r.status_code)
print(r.text)

200
[
  {
    "Year":2019,
    "Week":1,
    "Playoff Flag":false,
    "Team":"Justin",
    "Score":158.02,
    "Opp Score":113.2,
    "Win":1
  },
  {
    "Year":2019,
    "Week":1,
    "Playoff Flag":false,
    "Team":"Dante",
    "Score":107.92,
    "Opp Score":126.14,
    "Win":0
  },
  {
    "Year":2019,
    "Week":1,
    "Playoff Flag":false,
    "Team":"Michael",
    "Score":112.68,
    "Opp Score":134.32,
    "Win":0
  },
  {
    "Year":2019,
    "Week":1,
    "Playoff Flag":false,
    "Team":"Andrew",
    "Score":128.54,
    "Opp Score":141.26,
    "Win":0
  },
  {
    "Year":2019,
    "Week":1,
    "Playoff Flag":false,
    "Team":"Haris",
    "Score":135.8,
    "Opp Score":101.82,
    "Win":1
  },
  {
    "Year":2019,
    "Week":1,
    "Playoff Flag":false,
    "Team":"Noah ",
    "Score":153.86,
    "Opp Score":132.22,
    "Win":1
  },
  {
    "Year":2019,
    "Week":1,
    "Playoff Flag":false,
    "Team":"Klapp",
    "Score":113.2,
    "Opp Score":158.02,
    "Win":0
  },